In [8]:
from bs4 import BeautifulSoup
from xmlr import xmliter
import tqdm
import re
import json
import pickle

In [13]:
xml = "./PostLinks.V1.3.xml"
# {'@Id': '11217966', '@CreationDate': '2013-02-18T03:03:17.867', 
# '@PostId': '114876', '@RelatedPostId': '21688', '@LinkTypeId': '3'}

# the link type: [1:related, 3:duplicate]
duplicate_map = {}
for attr in tqdm.tqdm(xmliter(xml, "row")):
    if attr["@LinkTypeId"] == "3":
        pid = attr["@PostId"]
        dpid = attr["@RelatedPostId"]
        duplicate_map[pid] = dpid
    elif attr["@LinkTypeId"] == "1":
        pass
    else:
        assert False

pickle.dump(duplicate_map, open("duplicate_map.pkl", 'wb'))

257022it [00:01, 168370.41it/s]


In [16]:
xml = "./Posts.V1.3.xml"
question_posts = {}
answer_posts = {}
skip_count = 0
for attr in tqdm.tqdm(xmliter(xml, "row")):
#question post
# {'@Id': '1', '@PostTypeId': '1', 
#'@CreationDate': '2010-07-20T19:09:27.200', 
#'@ViewCount': '8695', '@Score': '155', '@CommentCount': '0', 
#'@OwnerUserId': '10', 
#'@Title': 'What Does it Really Mean to Have Different Kinds of Infinities?', 
#'@Body': '<p>Can someone explain to me how there can be different kinds of infinities?</p>  <p>I was reading "<a href="http://en.wikipedia.org/wiki/The_Man_Who_Loved_Only_Numbers" rel="noreferrer">The man who loved only numbers</a>" by <a href="http://en.wikipedia.org/wiki/Paul_Hoffman_(science_writer)" rel="noreferrer">Paul Hoffman</a> and came across the concept of countable and uncountable infinities, but they\'re only words to me.</p>  <p>Any help would be appreciated.</p> ', 
#'@AnswerCount': '9', '@AcceptedAnswerId': '9', '@Tags': '<elementary-set-theory><intuition><infinity><faq>'}

#answer post
#<row Id="4" PostTypeId="2" 
#CreationDate="2010-07-20T19:14:10.603" Score="11" CommentCount="1" 
#OwnerUserId="31" 
#Body="&lt;p&gt;&lt;a href=&quot;http://www.bbc.co.uk/podcasts/series/moreorless&quot; rel=&quot;noreferrer&quot;&gt;More or Less&lt;/a&gt; is a BBC Radio 4 programme about maths and statistics in the news, and there is a free podcast. It's presented by &lt;a href=&quot;http://timharford.com/&quot; rel=&quot;noreferrer&quot;&gt;Tim Harford&lt;/a&gt;, the Undercover Economist from the &lt;a href=&quot;http://www.ft.com/home/uk&quot; rel=&quot;noreferrer&quot;&gt;Financial Times&lt;/a&gt;.&lt;/p&gt; " 
#ParentId="3"/>
    attrs = {}
    for k in attr:
        attrs[k.replace("@","")] = attr[k]
    ptype = attrs["PostTypeId"]
    if ptype == "1": # Question
        qid = attrs["Id"]
        question_posts[qid] = {
            "accept_aid":attrs["AcceptedAnswerId"] if "AcceptedAnswerId" in attrs else None,
            "answers": []
        }
    elif ptype == "2": # Answer
        aid = attrs["Id"]
        answer_posts[aid] = {
            "body": attrs["Body"],
            "pid": attrs["ParentId"],
            "score": attrs["Score"],
        }
    else:
        skip_count += 1
        
print(skip_count)

2466080it [01:14, 33314.95it/s]

0


In [17]:
pickle.dump(answer_posts, open("answer_posts.pkl", 'wb'))
pickle.dump(question_posts, open("question_posts.pkl", 'wb'))

In [34]:
answer_posts = pickle.load(open("answer_posts.pkl", 'rb'))

In [37]:
question_posts = pickle.load(open("question_posts.pkl", 'rb'))

In [18]:
missing_qid = []
for aid in tqdm.tqdm(answer_posts):
#     print(ap)
    qid = answer_posts[aid]["pid"]
    if qid in question_posts:
        question_posts[qid]["answers"].append(aid)
    else:
        missing_qid.append(qid)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1445495/1445495 [00:01<00:00, 1433633.01it/s]


In [19]:
pickle.dump(question_posts, open("question_posts.pkl", 'wb'))

In [22]:
for qid in tqdm.tqdm(question_posts):
    # for all questions without accepted answer, find its duplicate qustion accept answer if possible.
    if not question_posts[qid]["accept_aid"] and qid in duplicate_map:
        dqid = duplicate_map[qid]
        if dqid in question_posts and question_posts[dqid]["accept_aid"]:
            question_posts[qid]["accept_aid"] = question_posts[dqid]["accept_aid"]
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1020585/1020585 [00:00<00:00, 2091215.94it/s]


In [31]:
for qid in tqdm.tqdm(question_posts):
    max_score = 0
    question_posts[qid]["best_answer"] = None
    for aid in question_posts[qid]["answers"]:
        score = float(answer_posts[aid]["score"])
        if score > max_score:
            question_posts[qid]["best_answer"] = aid
            max_score = score
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1020585/1020585 [00:01<00:00, 751830.33it/s]


In [32]:
pickle.dump(question_posts, open("question_posts.pkl", 'wb'))


In [33]:
aid2best_aid = {}
for aid in tqdm.tqdm(answer_posts):
    qid = answer_posts[aid]["pid"]
    if qid in question_posts:
        best_aid = question_posts[qid]["accept_aid"] or question_posts[qid]["best_answer"]
        aid2best_aid[aid] = best_aid if (best_aid and len(best_aid) > 0) else None
        if aid2best_aid[aid] is None or aid2best_aid[aid] not in answer_posts:
            continue
        if float(answer_posts[aid2best_aid[aid]]["score"]) < 2:
            aid2best_aid[aid] = None
    else:
        aid2best_aid[aid] = None


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1445495/1445495 [00:02<00:00, 706756.65it/s]


In [34]:
pickle.dump(aid2best_aid, open("aid2bestgt1_aid.pkl", 'wb'))